In [1]:
import os,glob
import pandas as pd
import nltk

In [2]:


doc = open('files_langeslag_batch2/PsGlA.txt').read().splitlines()





In [4]:
corpus_index = []
corpus_book = []
corpus_english = []
corpus_latin = []

for i in glob.glob('files_langeslag_batch2/*txt'):
    doc = open(i).read().splitlines()
    for index,sentence in enumerate(doc):
        splitted = sentence.split("||") 
        if len(splitted)==3:
            file_name = i.split("/")[1].split(".")[0]
            index = splitted[0].strip()
            english = splitted[1].strip()
            latin = splitted[2].strip()
            if bool(index) and bool(english) and bool(latin):
                corpus_index.append(index)
                corpus_book.append(file_name)
                corpus_english.append(english)
                corpus_latin.append(latin)

df = pd.DataFrame({"index":corpus_index,"book":corpus_book,
"english":corpus_english,"latin":corpus_latin})

## Drop duplicates on index

In [26]:
df.shape

(26562, 4)

In [27]:
df_nodups.shape

(2592, 4)

In [5]:
df_nodups = df.drop_duplicates(subset=["index"])

In [6]:
from sklearn.feature_extraction.text import CountVectorizer

In [36]:
# Count Vectorizer
vect = CountVectorizer()  
vects = vect.fit_transform(df_nodups.english)

td = pd.DataFrame(vects.todense())
td.columns = vect.get_feature_names()
term_document_matrix_english = td.copy()


/home/sago/miniconda3/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [41]:
term_document_matrix_english.sample(n=5,axis=1,ignore_index=False)

,afundenyssum,gegyrede,awurpon,gehæfde,standende
0,0,0,0,0,0
1,0,0,0,0,0
2,0,0,0,0,0
3,0,0,0,0,0
4,0,0,0,0,0
...,...,...,...,...,...
2587,0,0,0,0,0
2588,0,0,0,0,0
2589,0,0,0,0,0
2590,0,0,0,0,0


In [38]:
# Count Vectorizer
vect = CountVectorizer()  
vects = vect.fit_transform(df_nodups.latin)

td = pd.DataFrame(vects.todense())
td.columns = vect.get_feature_names()
term_document_matrix_latin = td.copy()

/home/sago/miniconda3/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [42]:
term_document_matrix_latin.sample(n=5,axis=1,ignore_index=False)

,inferno,folium,consolatus,immutata,apparuerint
0,0,0,0,0,0
1,0,1,0,0,0
2,0,0,0,0,0
3,0,0,0,0,0
4,0,0,0,0,0
...,...,...,...,...,...
2587,0,0,0,0,0
2588,0,0,0,0,0
2589,0,0,0,0,0
2590,0,0,0,0,0


In [43]:
sample_df_english = term_document_matrix_english.sample(n=100,axis=1,ignore_index=False)
sample_df_latin = term_document_matrix_latin.sample(n=100,axis=1,ignore_index=False)

In [46]:
df_corr=sample_df_english.corrwith(sample_df_latin)
df_corr = df_corr.unstack().sort_values(kind="quicksort",ascending=False)


In [51]:
df_corr = pd.concat([sample_df_english, sample_df_latin], axis=1, keys=['sample_df_english', 'sample_df_latin']).corr().loc['sample_df_english', 'sample_df_latin']
df_corr = df_corr.unstack().sort_values(kind="quicksort",ascending=False)

In [53]:
df_corr

conculcabis   weormcynna      1.000000
disposui      dauide          0.706970
longitudinem  langsumnysse    0.706834
siti          scæncton        0.577127
plantasti     ægiptalande     0.577127
                                ...   
in            deme           -0.023124
              oferwunnon     -0.028326
              rammas         -0.028326
              wuldres        -0.029925
              gemærsod       -0.036583
Length: 10000, dtype: float64

In [24]:
df_corr.loc[lambda x : x< 1]

æslitendras  forgægende     0.706970
forgægende   æslitendras    0.706970
dege         næht           0.577127
halu         dege           0.577127
næht         dege           0.577127
                              ...   
ymbhwyrfte   æfter         -0.009209
asende       æfter         -0.009634
æfter        asende        -0.009634
si           æfter         -0.012410
æfter        si            -0.012410
Length: 9898, dtype: float64

In [70]:
td = pd.DataFrame(vects.todense()).iloc[:5]  
td.columns = vect.get_feature_names()
term_document_matrix = td.T
term_document_matrix.columns = ['Doc '+str(i) for i in range(1, 6)]
term_document_matrix['total_count'] = term_document_matrix.sum(axis=1)

# Top 25 words 
term_document_matrix = term_document_matrix.sort_values(by ='total_count',ascending=False)[:25] 

# Print the first 10 rows 
print(term_document_matrix.drop(columns=['total_count']).head(10))

           Doc 1  Doc 2  Doc 3  Doc 4  Doc 5
his            2      3      0      0      0
on             2      1      0      2      0
swa            0      2      2      0      0
ne             0      1      1      2      0
he             1      2      0      0      0
þa             0      0      1      2      0
þe             0      2      0      0      1
rihtwisra      0      0      0      1      1
swaswa         0      1      1      0      0
þæt            0      2      0      0      0


/home/sago/miniconda3/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [55]:
len(corpus_english)

26562

In [57]:
df.to_excel("test.xlsx")

In [13]:
unique_index = set(cospus_index)

In [22]:
unique_index

{'58.1 ',
 '36.27 ',
 '59.6 ',
 '104.19 ',
 '18.13 ',
 '70.5 ',
 '109.7 ',
 '51.4 ',
 '117.10 ',
 '64.12 ',
 '77.63 ',
 '11.6 ',
 '118.80 ',
 '83.5 ',
 '138.21 ',
 '108.3 ',
 '88.14 ',
 '88.52 ',
 '112.3 ',
 '82.18 ',
 '38.7 ',
 '34.22 ',
 '51.11 ',
 '67.36 ',
 '135.27 ',
 '26.1 ',
 '121.6 ',
 '143.16 ',
 '140.7 ',
 '117.18 ',
 '73.11 ',
 '53.5 ',
 '117.13 ',
 '77.15 ',
 '135.1 ',
 '9.13 ',
 '27.4 ',
 '103.30 ',
 '85.10 ',
 '73.19 ',
 '27.6 ',
 '16.6 ',
 '34.3 ',
 '43.25 ',
 '144.1 ',
 '46.10 ',
 '134.3 ',
 '33.14 ',
 '73.14 ',
 '72.19 ',
 '9.5 ',
 '118.138 ',
 '142.9 ',
 '46.7 ',
 '86.6 ',
 '146.9 ',
 '13.12 ',
 '9.4 ',
 '50.8 ',
 '91.2 ',
 '50.13 ',
 '6.6 ',
 '106.21 ',
 '1.1 ',
 '118.35 ',
 '71.1 ',
 '118.81 ',
 '150.1 ',
 '108.13 ',
 '67.11 ',
 '29.10 ',
 '9.18 ',
 '49.16 ',
 '104.14 ',
 '67.23 ',
 '65.3 ',
 '17.30 ',
 '9.27 ',
 '33.3 ',
 '71.10 ',
 '59.2 ',
 '118.89 ',
 '39.11 ',
 '37.22 ',
 '117.14 ',
 '15.4 ',
 '42.2 ',
 '20.4 ',
 '64.10 ',
 '88.53 ',
 '108.5 ',
 '149.7 ',
 '134

In [1]:
PsGlA = open('files_langeslag_batch2/PsGlA.txt').read().splitlines()

In [11]:
PsGlA_latin[0]

' Qui habitat in caelis inridebit eos et dominus subsannabit eos.'

In [10]:
PsGlA_english[0]

' Se eardað in heofenum bismerad hie & dryhten hyspeð hie '

In [20]:
from cltk.languages.pipelines import LatinPipeline

/home/sago/miniconda3/lib/python3.9/site-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


In [21]:
a_pipeline = LatinPipeline()

In [27]:
a_pipeline.add_process(PsGlA_latin[0])

In [28]:
from cltk.core.data_types import Doc

In [29]:
Doc(raw=PsGlA_latin[0])

Doc(language=None, words=None, pipeline=None, raw=' Qui habitat in caelis inridebit eos et dominus subsannabit eos.', normalized_text=None)

## Base notebook https://github.com/cltk/cltk/blob/master/notebooks/CLTK%20data%20types.ipynb

In [30]:
from cltk import NLP
cltk_nlp = NLP(language="lat")
cltk_doc = cltk_nlp.analyze(text=PsGlA_latin[0])

‎𐤀 CLTK version '1.1.6'.
Pipeline for language 'Latin' (ISO: 'lat'): `LatinNormalizeProcess`, `LatinStanzaProcess`, `LatinEmbeddingsProcess`, `StopsProcess`, `LatinLexiconProcess`.
CLTK message: This part of the CLTK depends upon the Stanza NLP library.
CLTK message: Allow download of Stanza models to ``/home/sago/stanza_resources/la/tokenize/ittb.pt``? [Y/n] 


2022-11-23 10:07:25 INFO: Downloading these customized packages for language: la (Latin)...
| Processor | Package |
-----------------------
| tokenize  | ittb    |
| pos       | ittb    |
| lemma     | ittb    |
| depparse  | ittb    |
| pretrain  | ittb    |

2022-11-23 10:07:51 INFO: Finished downloading models and saved to /home/sago/stanza_resources.


CLTK message: This part of the CLTK depends upon word embedding models from the Fasttext project.
Do you want to download file 'https://dl.fbaipublicfiles.com/fasttext/vectors-wiki/wiki.la.vec' to '/home/sago/cltk_data/lat/embeddings/fasttext/wiki.la.vec'? [Y/n] 


100%|██████████| 364M/364M [00:42<00:00, 8.56MiB/s] 


This part of the CLTK depends upon models from the CLTK project.
Do you want to download 'https://github.com/cltk/lat_models_cltk' to '~/cltk_data/lat'? [Y/n] 
This part of the CLTK depends upon Lewis's *An Elementary Latin Dictionary* (1890).
Do you want to download this? [Y/n] 


In [33]:
cltk_doc.morphosyntactic_features 

[{Case: [nominative], Gender: [masculine], Number: [singular], PrononimalType: [relative]},
 {Aspect: [imperfective], Mood: [indicative], Number: [singular], Person: [third], Tense: [present], VerbForm: [finite], Voice: [active]},
 {AdpositionalType: [preposition]},
 {Case: [ablative], Gender: [masculine], Number: [plural]},
 {Aspect: [imperfective], Mood: [indicative], Number: [singular], Person: [third], Tense: [future], VerbForm: [finite], Voice: [active]},
 {Case: [accusative], Gender: [masculine], Number: [plural], PrononimalType: [demonstrative]},
 {},
 {Case: [nominative], Gender: [masculine], Number: [singular]},
 {Aspect: [imperfective], Mood: [indicative], Number: [singular], Person: [third], Tense: [future], VerbForm: [finite], Voice: [active]},
 {Case: [accusative], Gender: [masculine], Number: [plural], PrononimalType: [demonstrative]},
 {}]

In [34]:
cltk_doc.lemmata

['Qui',
 'habito',
 'in',
 'caelum',
 'inrido',
 'is',
 'et',
 'dominus',
 'subsanno',
 'is',
 '.']

In [40]:
cltk_nlp_english = NLP(language="ang")
cltk_doc_english = cltk_nlp_english.analyze(text=PsGlA_english[0])

‎𐤀 CLTK version '1.1.6'.
Pipeline for language 'Old English (ca. 450-1100)' (ISO: 'ang'): `MultilingualTokenizationProcess`, `OldEnglishLemmatizationProcess`, `OldEnglishEmbeddingsProcess`, `StopsProcess`.
This part of the CLTK depends upon models from the CLTK project.
Do you want to download 'https://github.com/cltk/ang_models_cltk' to '~/cltk_data/ang'? [Y/n] 
CLTK message: This part of the CLTK depends upon word embedding models from the Fasttext project.
Do you want to download file 'https://dl.fbaipublicfiles.com/fasttext/vectors-wiki/wiki.ang.vec' to '/home/sago/cltk_data/ang/embeddings/fasttext/wiki.ang.vec'? [Y/n] 


100%|██████████| 34.0M/34.0M [00:03<00:00, 9.47MiB/s]


In [42]:
cltk_doc_english.tokens

['Se',
 'eardað',
 'in',
 'heofenum',
 'bismerad',
 'hie',
 '&',
 'dryhten',
 'hyspeð',
 'hie']

In [41]:
cltk_doc_english.tokens_stops_filtered

['Se',
 'eardað',
 'heofenum',
 'bismerad',
 'hie',
 '&',
 'dryhten',
 'hyspeð',
 'hie']